In [15]:
import snowflake.connector
from dotenv import dotenv_values

# Loads into a dictionary
env_vars = dotenv_values(".env")

# Connect to Snowflake
conn = snowflake.connector.connect(
    user=env_vars['username'],
    password= env_vars['password'],
    account=env_vars['account'],  
    
)

print("✅ Connected to Snowflake!")
cursor = conn.cursor()


✅ Connected to Snowflake!


In [16]:
cursor.execute('use role accountadmin')


In [17]:
#push app to stage


cursor.execute("""CREATE OR REPLACE STAGE sandiego_ai.muni_code.search_app
               DIRECTORY = (ENABLE = TRUE)
    ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE')
  COMMENT = 'Stage for streamlit app'""")

cursor.execute("""
PUT file://search/streamlit_app.py 
@sandiego_ai.muni_code.search_app/search
OVERWRITE = TRUE 
AUTO_COMPRESS = FALSE
""")


cursor.execute("""
PUT file://search/environment.yml 
@sandiego_ai.muni_code.search_app/search
OVERWRITE = TRUE 
AUTO_COMPRESS = FALSE
""")


cursor.execute("""
CREATE OR REPLACE STREAMLIT sandiego_ai.muni_code.muni_code_search
  ROOT_LOCATION = '@sandiego_ai.muni_code.search_app/search'
  MAIN_FILE = 'streamlit_app.py'
  QUERY_WAREHOUSE = compute_wh
  TITLE = 'San Diego Municipal Code Search';


""")


In [18]:
#rag model

cursor.execute("""
PUT file://rag/streamlit_rag.py 
@sandiego_ai.muni_code.search_app/rag
OVERWRITE = TRUE 
AUTO_COMPRESS = FALSE
""")


cursor.execute("""
PUT file://rag/environment.yml 
@sandiego_ai.muni_code.search_app/rag
OVERWRITE = TRUE 
AUTO_COMPRESS = FALSE
""")


cursor.execute("""
CREATE OR REPLACE STREAMLIT sandiego_ai.muni_code.muni_code_rag_search
  ROOT_LOCATION = '@sandiego_ai.muni_code.search_app/rag'
  MAIN_FILE = 'streamlit_rag.py'
  QUERY_WAREHOUSE = compute_wh
  TITLE = 'San Diego Municipal Code Rag Search';


""")